In [12]:
!pip install nvcc4jupyter

In [13]:
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [17]:
%%cuda
#include <cuda.h>
#include <cuda_runtime.h>
#include <iostream>
#define N 100
using namespace std;

// DEVICE: Kernel para multiplicación de matrices
__global__ void matrixMultiplicationKernel(float* A, float* B, float* C, int n) {
    // Índices del hilo en la matriz resultado
    int row = blockIdx.y * blockDim.y + threadIdx.y; // Índice de fila
    int col = blockIdx.x * blockDim.x + threadIdx.x; // Índice de columna

    // Asegurarse de que los índices estén dentro de los límites
    if (row < n && col < n) {
        float value = 0.0;

        // Cálculo del producto escalar para la celda (row, col)
        for (int k = 0; k < n; ++k) {
            value += A[row * n + k] * B[k * n + col];
        }

        // Almacena el valor calculado en la matriz resultado
        C[row * n + col] = value;
    }
}

// HOST
int main() {
    int matrixSize = N * N * sizeof(float); // Tamaño total de una matriz

    // Reservar memoria para las matrices en el host
    float* A = new float[N * N];
    float* B = new float[N * N];
    float* C = new float[N * N]; // Resultado

    // Inicializar las matrices A y B en el host
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            A[i * N + j] = i + j;       // Elementos de A
            B[i * N + j] = i - j;       // Elementos de B
        }
    }

    // Reservar memoria en la GPU para las matrices
    float *d_A, *d_B, *d_C;
    cudaMalloc((void**)&d_A, matrixSize);
    cudaMalloc((void**)&d_B, matrixSize);
    cudaMalloc((void**)&d_C, matrixSize);

    // Copiar las matrices A y B del host al device
    cudaMemcpy(d_A, A, matrixSize, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, matrixSize, cudaMemcpyHostToDevice);

    // Configuración de bloques e hilos
    dim3 threadsPerBlock(16, 16); // 16x16 hilos por bloque
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Llamada al kernel
    matrixMultiplicationKernel<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, N);

    // Sincronizar la GPU
    cudaDeviceSynchronize();

    // Copiar el resultado de la GPU al host
    cudaMemcpy(C, d_C, matrixSize, cudaMemcpyDeviceToHost);

    // Mostrar una parte de la matriz resultado
    cout << "Matriz Resultado (primeros 10x10 elementos):" << endl;
    for (int i = 0; i < 10; ++i) {
        for (int j = 0; j < 10; ++j) {
            cout << C[i * N + j] << "\t";
        }
        cout << endl;
    }

    // Liberar memoria en el device
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Liberar memoria en el host
    delete[] A;
    delete[] B;
    delete[] C;

    return 0;
}


Matriz Resultado (primeros 10x10 elementos):
328350	323400	318450	313500	308550	303600	298650	293700	288750	283800	
333300	328250	323200	318150	313100	308050	303000	297950	292900	287850	
338250	333100	327950	322800	317650	312500	307350	302200	297050	291900	
343200	337950	332700	327450	322200	316950	311700	306450	301200	295950	
348150	342800	337450	332100	326750	321400	316050	310700	305350	300000	
353100	347650	342200	336750	331300	325850	320400	314950	309500	304050	
358050	352500	346950	341400	335850	330300	324750	319200	313650	308100	
363000	357350	351700	346050	340400	334750	329100	323450	317800	312150	
367950	362200	356450	350700	344950	339200	333450	327700	321950	316200	
372900	367050	361200	355350	349500	343650	337800	331950	326100	320250	

